# Fine-tune a YOLOv8 detection model

This notebook will train a [Yolov8](https://github.com/ultralytics/ultralytics) model for tank detection using publicly available annotated images of tanks.

As the notebook will run the training with `PyTorch`, it is recommended to have GPUs available. If running in Google Colab, go to Edit > Notebook settings and select GPU hardware acceleration.

This notebook expects that the dataset created in the notebook [01_Dataset](./01_Dataset.ipynb) has been saved to disk.

### Setup

To start, check GPU support.

In [1]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.3.1+cu121 (NVIDIA RTX A6000)


### Load dataset

We'll first load the dataset created on disk.

In [2]:
from pathlib import Path

dataset_dir = Path() / "dataset"

In [3]:
import fiftyone as fo

# The splits to load
splits = ["train", "val", "test"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset()
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

 100% |███████████████| 2270/2270 [2.0s elapsed, 0s remaining, 1.2K samples/s]      
 100% |█████████████████| 174/174 [153.4ms elapsed, 0s remaining, 1.2K samples/s]     
 100% |█████████████████| 173/173 [150.0ms elapsed, 0s remaining, 1.2K samples/s]     


In [4]:
session = fo.launch_app(dataset, auto=False)

Connected to FiftyOne on port 5151 at localhost.
If you are not connecting to a remote session, you may need to start a new session and specify a port
Session launched. Run `session.show()` to open the App in a cell output.


In [5]:
session.open_tab()

<IPython.core.display.Javascript object>

### Train a YoloV8 model

We can now train a YoloV8 base model on our exported dataset. We use the `yolov8n.pt` (nano) model, which is the smallest, but larger models are available from ultralytics. Simply replace `yolov8n.pt` with the [right model](https://docs.ultralytics.com/models/yolov8/#key-features).

In [6]:
from adomvi.yolo.yolo import train

train("yolov8m.pt", data=dataset_dir / "dataset.yaml", batch=64, device=[0, 1])

Ultralytics YOLOv8.2.31 🚀 Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
                                                            CUDA:1 (NVIDIA RTX A6000, 48655MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=dataset/dataset.yaml, epochs=60, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

train: Scanning /home/ukemkata/workspace/adomvi2/notebooks/dataset/labels/train... 2270 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2270/2270 [00:02<00:00, 958.15it/s] 


train: New cache created: /home/ukemkata/workspace/adomvi2/notebooks/dataset/labels/train.cache


val: Scanning /home/ukemkata/workspace/adomvi2/notebooks/dataset/labels/val... 174 images, 0 backgrounds, 0 corrupt: 100%|██████████| 174/174 [00:00<00:00, 1629.46it/s]


val: New cache created: /home/ukemkata/workspace/adomvi2/notebooks/dataset/labels/val.cache
Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 16 dataloader workers
Logging results to runs/detect/train2
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        174        263      0.879      0.804      0.879      0.681

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      60/60      13.5G     0.3507     0.2314     0.9565         18        640: 100%|██████████| 36/36 [00:11<00:00,  3.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.23it/s]


                   all        174        263      0.884      0.814      0.879      0.678

60 epochs completed in 0.221 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 52.0MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.31 🚀 Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
                                                            CUDA:1 (NVIDIA RTX A6000, 48655MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.07it/s]


                   all        174        263      0.842      0.833      0.879      0.686
Speed: 0.1ms preprocess, 2.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train2


### Use the fine-tuned model to predict detections

Once our model is trained, we can simply use it to predict detections on our test dataset. Simply pass the path to the best model weights and the directory containing the test images to the `predict` function.

In [7]:
from pathlib import Path
from adomvi.yolo.yolo import predict

best_model = Path() / "runs/detect/train/weights/best.pt"
results = predict(best_model, source=dataset_dir / "images/test")


image 1/173 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/00741fb6fb4c7209.jpg: 512x640 1 tank, 151.0ms
image 2/173 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/01d517af269faac1.jpg: 416x640 1 tank, 147.5ms
image 3/173 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/021d9739adae9f9c.jpg: 480x640 1 tank, 152.2ms
image 4/173 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/030eab769f338ba7.jpg: 352x640 1 tank, 88.5ms
image 5/173 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/043eccfe44f4ae5c.jpg: 480x640 2 tanks, 5.8ms
image 6/173 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/04deb749185941bf.jpg: 448x640 3 tanks, 87.9ms
image 7/173 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/05304464737d93eb.jpg: 480x640 1 tank, 5.4ms
image 8/173 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/06151edcba0f9cf9.jpg: 384x640 2 tanks, 87.7ms
image 9/173 /home/ukemkata/workspac

### Evaluate model predictions

We can first evaluate our model by loading the predictions in a Fiftyone dataset view of our test data. We select the images tagged as part of the test set in our dataset.

In [8]:
# The test split of the dataset
test_view = dataset.match_tags("test")

Then we can load the model predictions as fields of the images in our test view.

In [10]:
from pathlib import Path
from adomvi.yolo.utils import add_yolo_detections

prediction_field = "yolov8"
predictions_dir = Path() / "runs/detect/predict/labels"
add_yolo_detections(test_view, prediction_field=prediction_field, predictions_dir=predictions_dir, class_list=["tank"])

Now we can visualize these YOLOv8 model predictions on the samples in our dataset in the FiftyOne app:

In [11]:
session.open_tab()

<IPython.core.display.Javascript object>

Finally, we can evaluate our model's predictions and print the mAP.

In [12]:
detection_results = test_view.evaluate_detections(
    prediction_field, 
    eval_key="eval",
    compute_mAP=True,
    gt_field="ground_truth",
)

Evaluating detections...
 100% |█████████████████| 173/173 [255.2ms elapsed, 0s remaining, 677.9 samples/s]      
Performing IoU sweep...
 100% |█████████████████| 173/173 [298.7ms elapsed, 0s remaining, 579.2 samples/s]      


In [13]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

mAP = 0.05006000600060006


In [14]:
detection_results.print_report()

              precision    recall  f1-score   support

        tank       0.76      0.06      0.11       264

   micro avg       0.76      0.06      0.11       264
   macro avg       0.76      0.06      0.11       264
weighted avg       0.76      0.06      0.11       264

